# EEM: Based on the examples in the book "Finite-element method for beams".

By Johan Blaauwendraad http://www.delftacademicpress.nl/f040.php (in Dutch).

This Tutorial_02 notebook uses the figure 1.1 example in EEM and explains in detail how the FE4_1 skeleton program in the PtFEM package works.

Tutorial_02 continues where Tutorial_01 ends. Repeat the initial steps because later on in the explanations we need m.kv, the stiffnes matrix in Skyline format.

In [48]:
using PtFEM, DataFrames, Plots

In [49]:
struct_el = :Rod   # A 1-dimensional structural element that can only handle axial forces.
fin_el = :Line     # The type of finite element used to construct a mesh for the structural element;

In [50]:
l = 1.0       # Total length of structural element [m]
q = 5.0       # Distributed load [N/m]
N = 5        # Number of nodes
els = N - 1   # Number of finite elements
nod = 2       # Number of nodes per finite elements
nodof = 1     # Degrees of freedom for each node, just axial ;

In [51]:
data = Dict(
  # Rod(nxe, np_types, nip, fin_el(nod, nodof))
  :struc_el => getfield(Main, Symbol(struct_el))(els, 1, 1,
    getfield(Main, Symbol(fin_el))(nod, nodof)),
  :properties => [1.0e5;],
  :x_coords => 0.0:l/els:l,
  :support => [(N, [0])],
  :loaded_nodes => [(i, repeat([0.0], inner=nodof)) for i in 1:N],
  :eq_nodal_forces_and_moments => [(i, repeat([0.0], inner=nodof*nod)) for i in 1:els]
);

In [52]:
for node in 1:els
  data[:eq_nodal_forces_and_moments][node][2][1] = (1/2)*q*l/els
  data[:eq_nodal_forces_and_moments][node][2][2] = (1/2)*q*l/els
end  

In [53]:
for node in 1:N-1
  data[:loaded_nodes][node][2][1] += data[:eq_nodal_forces_and_moments][node][2][1]
  data[:loaded_nodes][node+1][2][1] += data[:eq_nodal_forces_and_moments][node][2][2]
end  

In [54]:
m = FE4_1(data);

There are 4 equations and the skyline storage is 7.



### Here starts the review of all steps in FE4_1

First couple of steps just go over the input data Dictionary and copies the entries to global variables. This is just for demonstration purposes. Inreality all of this is done inside the FE4_1 function.

In [55]:
if :struc_el in keys(data)
    struc_el = data[:struc_el]
end

PtFEM.Rod(4,1,1,PtFEM.Line(2,1))

In [56]:
ndim = 1
nst = struc_el.np_types;

In [57]:
fin_el = struc_el.fin_el

PtFEM.Line(2,1)

In [58]:
if typeof(fin_el) == Line # 1D finite element
    (nels, nn) = PtFEM.mesh_size(fin_el, struc_el.nxe)
    elseif typeof(fin_el) == Triangle || typeof(fin_el) == Quadrilateral # 2D finite elements
    (nels, nn) = mesh_size(fin_el, struc_el.nxe, struc_el.nye)
    elseif typeof(fin_el) == Hexahedron  # 3D finite elements
    (nels, nn) = mesh_size(fin_el, struc_el.nxe, struc_el.nye, struc_el.nze)
end

(4,5)

In [59]:
nodof = fin_el.nodof         # Degrees of freedom per node

1

In [60]:
ndof = fin_el.nod * nodof    # Degrees of freedom per fin_el, in this case each finite element has 2 nodes

2

In [61]:
penalty = 1e20                         # used to fix fixed_freedoms
if :penalty in keys(data)
    penalty = data[:penalty]
end

In [62]:
if :properties in keys(data)
    prop = zeros(size(data[:properties], 1), size(data[:properties], 2))
    for i in 1:size(data[:properties], 1)
        prop[i, :] = data[:properties][i, :]
    end
end

In [63]:
prop

1×1 Array{Float64,2}:
 100000.0

In [64]:
nf = ones(Int64, nodof, nn)
if :support in keys(data)
    for i in 1:size(data[:support], 1)
        nf[:, data[:support][i][1]] = data[:support][i][2]
    end
end
(size(nf), nf)

((1,5),
[1 1 … 1 0])

In [65]:
x_coords = zeros(nn)
if :x_coords in keys(data)
    x_coords = data[:x_coords]
end
  
y_coords = zeros(nn)
if :y_coords in keys(data)
    y_coords = data[:y_coords]
end
  
z_coords = zeros(nn)
if :z_coords in keys(data)
    z_coords = data[:z_coords]
end

etype = ones(Int64, nels)
if :etype in keys(data)
    etype = data[:etype]
end
x_coords

0.0:0.25:1.0

In [66]:
collect(x_coords)

5-element Array{Float64,1}:
 0.0 
 0.25
 0.5 
 0.75
 1.0 

In [67]:
#
# Initialize dynamic arrays stored in the FEM object
#

points = zeros(struc_el.nip, ndim)
g = zeros(Int64, ndof)
g_coord = zeros(ndim,nn)
fun = zeros(fin_el.nod)
coord = zeros(fin_el.nod, ndim)
gamma = zeros(nels)
jac = zeros(ndim, ndim)
g_num = zeros(Int64, fin_el.nod, nels)
der = zeros(ndim, fin_el.nod)
deriv = zeros(ndim, fin_el.nod)
bee = zeros(nst,ndof)
km = zeros(ndof, ndof)
mm = zeros(ndof, ndof)
gm = zeros(ndof, ndof)
kg = zeros(ndof, ndof)
eld = zeros(ndof)
weights = zeros(struc_el.nip)
g_g = zeros(Int64, ndof, nels)
num = zeros(Int64, fin_el.nod)
actions = zeros(nels, ndof)
displacements = zeros(size(nf, 1), ndim)
gc = ones(ndim, ndim)
dee = zeros(nst,nst)
sigma = zeros(nst)
axial = zeros(nels);

Ok, most arrays have been initialized. Time to start the real work. First determine the global numbering:

In [68]:
?PtFEM.formnf!

## formnf!

Returns nodal freedom numbering array nf

### Function

```julia
formnf!(nodof::Int64, nn::Int64, nf::Matrix{Int64})
```

### Arguments

```julia
* nodof::Int64       : Number of degrees of freedom for each node
* nn::Int64          : Number of nodes in mesh
* nf::Array{Int64,2} : Nodal freedom matrix (updated)
```


In [69]:
PtFEM.formnf!(nodof, nn, nf)
nf

1×5 Array{Int64,2}:
 1  2  3  4  0

Node N (= 10 in this example) is fixed, nodes 1 to 9 represents the degrees of freedom. We need 9 equations to solve for these 9 displacements.

In [70]:
neq = maximum(nf)

4

In [71]:
kdiag = zeros(Int64, neq);

In [72]:
ell = zeros(nels) # Used to hold element length
if :x_coords in keys(data)
    for i in 1:length(data[:x_coords])-1
        ell[i] = data[:x_coords][i+1] - data[:x_coords][i]
    end
end
ell

4-element Array{Float64,1}:
 0.25
 0.25
 0.25
 0.25

In [73]:
for i in 1:nels
    num = [i; i+1]
    PtFEM.num_to_g!(fin_el.nod, nodof, nn, ndof, num, nf, g)
    g_g[:, i] = g
    PtFEM.fkdiag!(ndof, neq, g, kdiag)
end

### Matrix g_g now holds the globally numbered degrees of freedom for the Line elements. Note that the 2nd node of element 9 is fixed.

In [74]:
g_g

2×4 Array{Int64,2}:
 1  2  3  4
 2  3  4  0

### Compute the locations in the skyline vector kv which form the diagonal elements of the full stiffness matrix.

In [75]:
for i in 2:neq
    kdiag[i] = kdiag[i] + kdiag[i-1]
end
kdiag

4-element Array{Int64,1}:
 1
 3
 5
 7

Above kdiag holds the indices of the diagonal elements of the stiffness matrix in the kv skyline vector.

To illustrate this, we'll borrow kv from the earlier call to FE4_1:

In [76]:
m.kv

7-element Array{Float64,1}:
  632.456
 -632.456
  632.456
 -632.456
  632.456
 -632.456
  632.456

The function ```fromSkyline(m.kv, m.kdiag)``` uses the indices in kdiag to reconstruct the (symmetrical) stiffness matrix. 

In [77]:
sm = fromSkyline(m.kv, m.kdiag)

4×4 Array{Float64,2}:
  632.456  -632.456     0.0       0.0  
 -632.456   632.456  -632.456     0.0  
    0.0    -632.456   632.456  -632.456
    0.0       0.0    -632.456   632.456

Julia has it's own sparse matrix representation. At some point the intenstion is to replace the skyline format with the Julia SparseArrays representation.

In [78]:
?AbstractSparseArray

search: AbstractSparseArray AbstractSparseMatrix AbstractSparseVector



No documentation found.

**Summary:**

```
abstract AbstractSparseArray{Tv,Ti,N} <: AbstractArray{Tv,N}
```

**Subtypes:**

```
Base.SparseArrays.CHOLMOD.Sparse{Tv<:Union{Complex{Float64},Float64}}
SparseMatrixCSC{Tv,Ti<:Integer}
SparseVector{Tv,Ti<:Integer}
```


In [79]:
sparse(sm)

4×4 sparse matrix with 10 Float64 nonzero entries:
	[1, 1]  =  632.456
	[2, 1]  =  -632.456
	[1, 2]  =  -632.456
	[2, 2]  =  632.456
	[3, 2]  =  -632.456
	[2, 3]  =  -632.456
	[3, 3]  =  632.456
	[4, 3]  =  -632.456
	[3, 4]  =  -632.456
	[4, 4]  =  632.456

In [80]:
kv = zeros(kdiag[neq])
gv = zeros(kdiag[neq])
print("There are $(neq) equations,")
println(" and the skyline storage is $(kdiag[neq]).\n")

There are 4 equations, and the skyline storage is 7.



In [81]:
loads = zeros(neq+1)
if :loaded_nodes in keys(data)
    for i in 1:size(data[:loaded_nodes], 1)
        loads[nf[:, data[:loaded_nodes][i][1]]+1] = 
            data[:loaded_nodes][i][2]
    end
end
nf

1×5 Array{Int64,2}:
 1  2  3  4  0

In [82]:
for i in 1:nels
    km = PtFEM.rod_km!(km, prop[etype[i], 1], ell[i])
    g = g_g[:, i]
    PtFEM.fsparv!(kv, km, g, kdiag)
end

In [83]:
km

2×2 Array{Float64,2}:
  400000.0  -400000.0
 -400000.0   400000.0

In [84]:
kv

7-element Array{Float64,1}:
  400000.0
 -400000.0
  800000.0
 -400000.0
  800000.0
 -400000.0
  800000.0

In [85]:
# Add radial stress if fin_el is 3d and axisymmetric
if ndim == 3 && struc_el.axisymmetric
    nst = 4
end

In [86]:
fixed_freedoms = 0
if :fixed_freedoms in keys(data)
    fixed_freedoms = size(data[:fixed_freedoms], 1)
end
no = zeros(Int64, fixed_freedoms)
node = zeros(Int64, fixed_freedoms)
sense = zeros(Int64, fixed_freedoms)
value = zeros(Float64, fixed_freedoms)
if :fixed_freedoms in keys(data) && fixed_freedoms > 0
    for i in 1:fixed_freedoms
      node[i] = data[:fixed_freedoms][i][1]
      sense[i] = data[:fixed_freedoms][i][2]
      no[i] = nf[sense[i], node[i]]
      value[i] = data[:fixed_freedoms][i][3]
    end
    kv[kdiag[no]] = kv[kdiag[no]] + penalty
    loads[no+1] = kv[kdiag[no]] .* value
end
loads

5-element Array{Float64,1}:
 0.625
 0.625
 1.25 
 1.25 
 1.25 

In [87]:
PtFEM.sparin!(kv, kdiag)

In [88]:
kv

7-element Array{Float64,1}:
  632.456
 -632.456
  632.456
 -632.456
  632.456
 -632.456
  632.456

In [89]:
loads[2:end] = PtFEM.spabac!(kv, loads[2:end], kdiag)
loads

5-element Array{Float64,1}:
 0.625     
 2.5e-5    
 2.34375e-5
 1.875e-5  
 1.09375e-5

In [90]:
displacements = zeros(size(nf))
for i in 1:size(displacements, 1)
    for j in 1:size(displacements, 2)
      if nf[i, j] > 0
        displacements[i,j] = loads[nf[i, j]+1]
      end
    end
end
displacements = displacements'

5×1 Array{Float64,2}:
 2.5e-5    
 2.34375e-5
 1.875e-5  
 1.09375e-5
 0.0       

In [91]:
loads[1] = 0.0
for i in 1:nels
    km = PtFEM.rod_km!(km, prop[etype[i], 1], ell[i])
    g = g_g[:, i]
    eld = loads[g+1]
    actions[i, :] = km * eld
end
actions

4×2 Array{Float64,2}:
 0.625  -0.625
 1.875  -1.875
 3.125  -3.125
 4.375  -4.375

In [92]:
m=FEM(struc_el, fin_el, ndim, nels, nst, ndof, nn, nodof, neq, penalty,
    etype, g, g_g, g_num, kdiag, nf, no, node, num, sense, actions, 
    bee, coord, gamma, dee, der, deriv, displacements, eld, fun, gc,
    g_coord, jac, km, mm, gm, kv, gv, loads, points, prop, sigma, value,
    weights, x_coords, y_coords, z_coords, axial);

In [93]:
m.displacements

5×1 Array{Float64,2}:
 2.5e-5    
 2.34375e-5
 1.875e-5  
 1.09375e-5
 0.0       

In [94]:
m.actions

4×2 Array{Float64,2}:
 0.625  -0.625
 1.875  -1.875
 3.125  -3.125
 4.375  -4.375